## For getting the goddamn outpuyt layer lmao

In [3]:
import time
from concurrent.futures import ThreadPoolExecutor
import glob
import librosa
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import os

print('start')
tf.experimental.numpy.experimental_enable_numpy_behavior()
START = time.time()
TERMINATE_TIME = START + 5300

# Load models and data
trained_model = tf.keras.layers.TFSMLayer('/kaggle/input/small_wavenet/other/default/1/wavenet_species_classifier', call_endpoint='serving_default')
primary_labels = pd.read_csv('/kaggle/input/birdclef-2025/sample_submission.csv').columns[1:].to_list()
primary_labels_map_bc = dict(zip(primary_labels, range(len(primary_labels))))
birdclassifier_indices = [primary_labels_map_bc.get(pl, len(primary_labels)) for pl in primary_labels]

def load_and_process_audio(path, sr=32000, duration=5.0, n_mels=64, n_mfcc=40, target_length=313):
    try:
        audio, _ = librosa.load(path, sr=sr, duration=duration)
        mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels)
        mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel_spectrogram), n_mfcc=n_mfcc)
        mfcc = mfcc.T

        if mfcc.shape[0] < target_length:
            padding = target_length - mfcc.shape[0]
            mfcc = np.pad(mfcc, ((0, padding), (0, 0)), mode='constant')
        elif mfcc.shape[0] > target_length:
            mfcc = mfcc[:target_length, :]

        return mfcc

    except Exception as e:
        print(f"Error processing {path}: {e}")
        return np.zeros((target_length, n_mfcc))

def process_ogg(ogg):
    _, fname, ss_id = ogg
    row_ids = [f'{ss_id}_{n}' for n in range(5, 65, 5)]

    if time.time() > TERMINATE_TIME:
        return row_ids, -1000 * np.ones((12, len(primary_labels)))

    try:
        segments = []
        for i in range(12):
            audio_segment = load_and_process_audio(fname, duration=5.0)
            segments.append(audio_segment)
        segments = np.array(segments)

        model_outputs_dict = trained_model(segments)
        print(f"Model output keys: {model_outputs_dict.keys()}") # Print the keys

        # Replace 'dense_7' with the correct output layer name
        if 'dense_7' in model_outputs_dict:
            model_outputs = model_outputs_dict['dense_7']
        elif len(model_outputs_dict) == 1: #if there is only one output, take the first value.
            model_outputs = list(model_outputs_dict.values())[0]
        else:
            print("Error: Could not find the output layer name.")
            return row_ids, -1000 * np.ones((12, len(primary_labels)))

        if model_outputs.shape != (12, len(primary_labels)):
            print(f"Error: Unexpected model_outputs shape: {model_outputs.shape}")
            return row_ids, -1000 * np.ones((12, len(primary_labels)))

        model_outputs = tf.pad(model_outputs, tf.constant([[0, 0], [0, 1]]))
        result = model_outputs[:, birdclassifier_indices]
        return row_ids, result

    except Exception as e:
        import traceback
        print(f"Error processing {ss_id}: {traceback.format_exc()}")
        return row_ids, -1000 * np.ones((12, len(primary_labels)))

oggs = [(n, ogg, re.search(r'/([^/]+)\.ogg$', ogg).group(1)) for n, ogg in enumerate(glob.glob('/kaggle/input/birdclef-2025/test_soundscapes/*.ogg') or sorted(glob.glob('/kaggle/input/birdclef-2025/train_soundscapes/*.ogg')))]

with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(process_ogg, oggs))

row_ids = [rid for rids, _ in results for rid in rids]
result = np.concatenate([res for _, res in results])

submission = pd.DataFrame(result, columns=primary_labels)
submission['row_id'] = row_ids
submission = submission[['row_id'] + primary_labels]

submission.to_csv('submission.csv', index=False)
display(submission.head(20))
display(submission.tail(20))

c:\Users\caspe\anaconda3\envs\BIRD\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Note: we can predict 206 species only!
Processing H02_20230420_074000
Processing H02_20230420_112000
Processing H02_20230420_154500
Processing H02_20230420_164000
H02_20230420_154500 audio loaded successfully.
H02_20230420_074000 audio loaded successfully.
H02_20230420_164000 audio loaded successfully.
H02_20230420_112000 audio loaded successfully.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/stepWARNING:tensorflow:6 out of the last 10 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000019C9B221440> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
H02_20230420_112000 prediction successful.
H02_20230420_164000 prediction successful.
H02_20230420_154500 prediction successful.
H02_20230420_074000 prediction successful.
Processing H02_20230420_223500
Processing H02_20230421_093000
Processing H02_20230421_113500
Processing H02_20230421_170000
H02_20230421_113500 audio loaded successfully.
H02_20230420_223500 audio loaded successfully.
H02_20230421_093000 audio loaded successfully.
H02_20230421_170000 audio loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/stepH02_20230420_223500 prediction successful.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
H02_20230421_093000 prediction successful.
H02_20230421_113500 prediction successful.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
H02_20230421_170000 prediction successful.

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,1.058064e-14,2.021399e-13,6.860196e-14,9.443593e-05,7.720923e-11,8.040034e-10,2.423830e-04,2.472993e-13,1.236679e-11,...,0.001738,0.000475,0.005852,0.002013,0.000848,0.004344,0.000342,0.003972,0.011952,0.015342
1,H02_20230420_074000_10,4.094188e-13,3.148623e-12,2.977367e-13,7.104839e-05,3.895167e-10,2.801041e-09,3.561054e-04,5.614933e-12,1.262840e-10,...,0.002640,0.000644,0.005060,0.002501,0.001038,0.006601,0.000696,0.005435,0.013388,0.014014
2,H02_20230420_074000_15,2.997793e-16,3.073062e-14,4.283420e-14,5.804573e-04,6.932635e-12,1.864347e-11,9.574467e-05,2.908594e-15,2.888200e-12,...,0.001337,0.000228,0.003317,0.002655,0.000651,0.005802,0.000362,0.003241,0.017126,0.017246
3,H02_20230420_074000_20,3.379855e-16,7.591475e-15,3.258494e-16,9.597653e-05,6.074030e-12,1.126817e-12,2.243897e-05,2.405853e-14,2.246956e-12,...,0.001489,0.000508,0.004022,0.001408,0.000795,0.004757,0.000356,0.002768,0.010953,0.020780
4,H02_20230420_074000_25,6.468638e-12,4.887330e-12,2.482350e-12,2.740313e-06,1.302862e-09,6.367248e-07,2.401972e-04,7.966555e-11,5.046919e-11,...,0.005556,0.001934,0.006844,0.002788,0.001692,0.003712,0.000511,0.011437,0.007560,0.007602
5,H02_20230420_074000_30,2.044854e-17,3.113748e-15,3.438129e-15,6.874574e-04,7.359661e-13,8.538578e-13,1.648535e-05,2.277696e-16,3.782487e-13,...,0.001110,0.000151,0.002409,0.002445,0.000517,0.006149,0.000324,0.002636,0.016411,0.019527
6,H02_20230420_074000_35,2.204586e-12,1.279463e-12,3.697116e-13,1.978014e-06,1.603033e-10,4.125150e-08,2.632888e-04,1.545053e-11,4.670283e-11,...,0.005987,0.001031,0.003587,0.003908,0.001100,0.008794,0.000876,0.010958,0.011354,0.007716
7,H02_20230420_074000_40,2.983635e-10,1.046637e-10,5.002510e-11,3.411792e-06,8.579669e-09,3.027824e-06,5.360240e-04,1.482932e-09,1.041250e-09,...,0.007542,0.002323,0.004986,0.004067,0.001871,0.006326,0.001037,0.015095,0.009597,0.006785
8,H02_20230420_074000_45,2.833850e-11,8.409459e-11,6.144842e-12,1.051727e-05,4.152100e-09,1.305329e-07,5.546161e-04,2.731820e-10,1.063898e-09,...,0.005875,0.001740,0.004756,0.003444,0.001742,0.007917,0.001226,0.011699,0.012982,0.010630
9,H02_20230420_074000_50,4.669584e-11,5.348714e-11,2.754736e-12,2.448770e-06,3.043548e-09,1.618999e-07,6.917003e-04,4.549365e-10,1.119123e-09,...,0.007062,0.001687,0.004369,0.003602,0.001642,0.009402,0.001406,0.012587,0.013458,0.009284


,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
76,H02_20230421_113500_25,6.795369e-11,1.580388e-12,5.640714e-13,5.787333e-10,2.371229e-10,8.253662e-05,0.000262,2.778386e-11,4.404581e-12,...,0.009598,0.003446,0.007015,0.001697,0.002085,0.001864,0.000628,0.023155,0.003281,0.002562
77,H02_20230421_113500_30,1.006896e-10,2.150296e-12,8.701907e-14,8.430571e-11,1.196691e-10,1.784658e-04,0.000251,1.702654e-11,1.468392e-12,...,0.010262,0.003146,0.005787,0.001455,0.002342,0.001905,0.000813,0.025293,0.003041,0.002883
78,H02_20230421_113500_35,2.118025e-10,7.744626e-12,5.049971e-13,1.187620e-09,4.942753e-10,1.756078e-04,0.000640,8.435339e-11,1.071619e-11,...,0.009344,0.002909,0.007620,0.001813,0.002200,0.002451,0.000799,0.022178,0.003973,0.003315
79,H02_20230421_113500_40,1.677786e-09,4.455919e-11,9.803726e-13,1.494171e-09,1.024599e-09,2.708714e-04,0.000704,2.044849e-10,2.703166e-11,...,0.008591,0.002733,0.006709,0.001838,0.002502,0.002889,0.001158,0.021654,0.004542,0.003834
80,H02_20230421_113500_45,9.029363e-11,2.193720e-12,1.136076e-12,1.509678e-10,2.762613e-10,2.702425e-04,0.000375,5.721019e-11,4.597447e-12,...,0.012650,0.004300,0.006823,0.001869,0.002270,0.001863,0.000698,0.030634,0.002666,0.001994
81,H02_20230421_113500_50,1.786909e-10,8.991242e-12,6.154996e-13,4.447521e-10,5.205176e-10,1.447704e-04,0.000476,8.240844e-11,1.153334e-11,...,0.010740,0.003421,0.007170,0.001825,0.002482,0.002324,0.000860,0.024684,0.003637,0.002958
82,H02_20230421_113500_55,2.327891e-09,6.304897e-11,2.933279e-12,6.492593e-10,1.575233e-09,1.439768e-04,0.000218,3.448756e-10,5.430829e-11,...,0.009557,0.003978,0.005697,0.001948,0.002728,0.002921,0.001222,0.024652,0.003736,0.002568
83,H02_20230421_113500_60,1.159137e-09,3.222588e-11,3.777090e-12,5.117763e-10,1.470567e-09,9.751632e-04,0.001184,3.145952e-10,3.105896e-11,...,0.010274,0.003548,0.008106,0.002019,0.002305,0.002411,0.000890,0.026108,0.003420,0.002544
84,H02_20230421_170000_5,8.006114e-11,2.625393e-11,2.518445e-07,2.126348e-06,2.847297e-09,5.002750e-07,0.000656,3.189548e-11,2.588109e-10,...,0.003458,0.001670,0.006166,0.007630,0.002466,0.005276,0.000684,0.016925,0.012998,0.002750
85,H02_20230421_170000_10,1.739936e-11,4.612870e-12,2.733167e-07,3.127786e-06,8.377503e-10,4.096885e-07,0.000544,6.588062e-12,8.139708e-11,...,0.003098,0.001304,0.005618,0.007673,0.002078,0.004569,0.000566,0.015051,0.013786,0.003067
